# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [6]:
stocks = pd.read_csv('starter_files/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [8]:
from starter_files.secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [20]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'MELSNANS)L  BA TLGCEGDQ(TRSOEAK/A', 'calculationPrice': 'close', 'open': 137.3, 'openTime': 1624318078405, 'openSource': 'ficlioaf', 'close': 137.29, 'closeTime': 1657060163013, 'closeSource': 'fafoiicl', 'high': 141.57, 'highTime': 1686703454681, 'highSource': 'diuaeideyc r 5t enl1emp', 'low': 134.79, 'lowTime': 1634063776331, 'lowSource': 'idtlee5n1r ped cmeauy i', 'latestPrice': 133.5, 'latestSource': 'Close', 'latestTime': 'January 29, 2021', 'latestUpdate': 1662520880666, 'latestVolume': 185436560, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 136.64, 'delayedPriceTime': 1681210558762, 'oddLotDelayedPrice': 132.04, 'oddLotDelayedPriceTime': 1649281138275, 'extendedPrice': 136.69, 'extendedChange': -0.19, 'extendedChangePercent': -0.00139, 'extendedPriceTime': 1684212642735, 'previousClose': 138.15, 'previousVolume': 145396784, 'change': -5.22, 'changePercent': -0.03803, 

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [31]:
price = data['latestPrice']
market_cap = data['marketCap']
print(f'Ticker: {symbol}\nPrice: ${price}\nMarket Cap: ${market_cap/1000000000000:.2f}T')

Ticker: AAPL
Price: $133.5
Market Cap: $2.22T


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [32]:
cols = ['Ticker', 'Stock Price', 'Market Capitalisation', 'Number of Shares to buy']
df = pd.DataFrame(columns = cols)
df

,Ticker,Stock Price,Market Capitalisation,Number of Shares to buy


In [35]:
df.append(
    pd.Series(
        [
            symbol, 
            price, 
            market_cap, 
            'N/A'
        ], index = cols
    ), ignore_index = True
)

,Ticker,Stock Price,Market Capitalisation,Number of Shares to buy
0,AAPL,133.5,2218578104457,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [39]:
df = pd.DataFrame(columns = cols)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    df = df.append(
        pd.Series([
            stock, 
            data['latestPrice'], 
            data['marketCap'], 
            'N/A'
        ], index = cols
        ), ignore_index = True
    )

In [40]:
df

,Ticker,Stock Price,Market Capitalisation,Number of Shares to buy
0,A,123.04,38001131555,N/A
1,AAL,17.98,10854696780,N/A
2,AAP,150.35,10446301228,N/A
3,AAPL,137.58,2227892586391,N/A
4,ABBV,103.04,188557355632,N/A
...,...,...,...,...
500,YUM,102.07,31794520779,N/A
501,ZBH,158.85,32723755596,N/A
502,ZBRA,398.52,21471226440,N/A
503,ZION,44.23,7366346124,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [42]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [48]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [57]:
df = pd.DataFrame(columns=cols)
for symbols in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for s in symbols.split(','):
        df = df.append(
            pd.Series([
                s, 
                data[s]['quote']['latestPrice'], 
                data[s]['quote']['marketCap'], 
                'N/A'
            ], index = cols
            ), ignore_index = True
        ) 
df

,Ticker,Stock Price,Market Capitalisation,Number of Shares to buy
0,A,121.69,38566234295,N/A
1,AAL,17.40,10886735464,N/A
2,AAP,152.36,10375276149,N/A
3,AAPL,136.12,2323423517807,N/A
4,ABBV,107.58,185844727094,N/A
...,...,...,...,...
500,YUM,104.08,30943528521,N/A
501,ZBH,157.74,33390693006,N/A
502,ZBRA,401.99,20857980100,N/A
503,ZION,45.00,7501023229,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [64]:
portfolio_size = input('Enter the value of your portfolio: $')

try: 
    val = float(portfolio_size)
except ValueError:
    print('Please make sure you are entering an integer.')
    portfolio_size = input('Enter the value of your portfolio: $')
    val = float(portfolio_size)

Enter the value of your portfolio: $1000000


In [65]:
position_size = val / len(df.index)
print(position_size)

1980.1980198019803


In [72]:
for i in range(len(df.index)):
    df.loc[i, 'Number of Shares to buy'] = math.floor(position_size / df.loc[i, 'Stock Price'])
df

,Ticker,Stock Price,Market Capitalisation,Number of Shares to buy
0,A,121.69,38566234295,16
1,AAL,17.40,10886735464,113
2,AAP,152.36,10375276149,12
3,AAPL,136.12,2323423517807,14
4,ABBV,107.58,185844727094,18
...,...,...,...,...
500,YUM,104.08,30943528521,19
501,ZBH,157.74,33390693006,12
502,ZBRA,401.99,20857980100,4
503,ZION,45.00,7501023229,44


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [117]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
df.to_excel(writer, 'Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [118]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color, 
        'bg_color': background_colour, 
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color, 
        'bg_color': background_colour, 
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color, 
        'bg_color': background_colour, 
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [119]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save

writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price',dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalisation', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of shares to buy', integer_format)


0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [120]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format], 
    'C': ['Market Capitalisation', dollar_format],
    'D': ['Number of shares to buy', integer_format]
}

for col in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column( f'{col}:{col}', 18, column_formats[col][1])
    writer.sheets['Recommended Trades'].write(f'{col}1', column_formats[col][0], column_formats[col][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [121]:
writer.save()